In [4]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 KB 500.6 kB/s eta 0:00:00a 0:00:01


In [6]:
import csv
import requests

api_key = '7a201835d4fca17fb0cb0bcd9372a722'
start_year = 2010
end_year = 2022
output_file = 'movies.csv'

with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Overview', 'Release Date', 'Vote Average', 'Vote Count', 'Runtime', 'Budget', 'Revenue', 'Popularity', 'Production Countries', 'Production Companies', 'Genres'])

    for year in range(start_year, end_year + 1):
        page = 1
        movie_count = 0

        while movie_count < 500:
            url = f'https://api.themoviedb.org/3/discover/movie?api_key={api_key}&language=en-US&sort_by=vote_count.desc&include_adult=true&include_video=false&page={page}&primary_release_year={year}&release_date.gte={year}-01-01&release_date.lte={year}-12-31'
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                results = data['results']

                if len(results) == 0:
                    break

                for movie in results:
                    if movie_count >= 500:
                        break

                    movie_id = movie['id']
                    details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US&append_to_response=runtime'
                    response = requests.get(details_url)

                    if response.status_code == 200:
                        details = response.json()

                        production_countries = [country['name'] for country in details['production_countries']]
                        if 'United States of America' in production_countries:
                            title = details['title']
                            overview = details['overview']
                            release_date = details['release_date']  
                            vote_average = details['vote_average']
                            vote_count = details['vote_count']
                            budget = details['budget'] if 'budget' in details else 'N/A'
                            revenue = details['revenue'] if 'revenue' in details else 'N/A'
                            popularity = details['popularity']
                            production_companies = [company['name'] for company in details['production_companies']]
                            genres = [genre['name'] for genre in details['genres']]
                            runtime = details['runtime']
                            writer.writerow([title, overview, release_date, vote_average, vote_count, runtime, budget, revenue, popularity, production_countries, production_companies, genres])
                            movie_count += 1
                    else:
                        print(f'Failed to retrieve data for movie {movie_id}')
            else:
                print(f'Failed to retrieve data for year {year} page {page}')

            page += 1

        if movie_count < 500:
            print(f'Unable to retrieve 500 movies for year {year}. Found only {movie_count} movies.')

print(f'Data written to {output_file}')

